### Init Context

In [4]:
from thetaray.api.context import init_context
import datetime
import yaml

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_human_trafficking/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']
context = init_context(execution_date=datetime.datetime(1970, 2, 1),
                       spark_conf=spark_config,
                       allow_type_changes=True,
                       delete_unused_columns=True,
                       spark_master='local[*]')

/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-07-29 12:48:08,464:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-07-29 12:48:08,472:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_human_trafficking_insights, but data encryption is disabled in deployment
2025-07-29 12:48:08,487:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_remittance_customer_insights, but data encryption is disabled in deployment
2025-07-29 12:48:08,497:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-07-29 12:48:08,535:WARNING:thetaray.common.logging:Encryption is enabled on dataset wrangling, but data encryption is disabl

Added `alias` successfully.
### Adding following columns to manager.demo_human_trafficking_ef metadata: algo_income_websites_cnt_rank, algo_ad_agency_out_cnt_rank, ad_agency_out_cnt, algo_income_websites_amt_rank, ad_agency_out_amt, algo_risky_expenses_cnt_rating, risky_expenses_cnt, income_websites_cnt, algo_income_websites_amt_rating, algo_ad_agency_out_amt_rating, algo_ad_agency_out_cnt_rating, algo_income_websites_cnt_rating, algo_risky_expenses_cnt_rank, income_websites_amt, algo_ad_agency_out_amt_rank ###


25/07/29 12:48:41 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/manager.db/demo_human_trafficking_ef
2025-07-29 12:48:44,072:INFO:thetaray.common.logging:=== Finished updating schema for Evaluation Flows on Minio ===


Added `alias` successfully.


### Imports

In [5]:
from thetaray.api.dataset import dataset_functions

from domains.demo_human_trafficking.datasets.customer_monthly import customer_monthly_dataset
from domains.demo_human_trafficking.datasets.customers import customers_dataset
from domains.demo_human_trafficking.datasets.transactions import transactions_dataset

from pyspark.sql import functions as f

### Data Gen

In [9]:

import pandas as pd
import numpy as np
import random, uuid
from datetime import datetime
from faker import Faker

faker = Faker()

# Constantes
HIGH_RISK_COUNTRIES = ['IR', 'KP', 'SY', 'SD', 'VE', 'SO', 'YE', 'CU', 'MM', 'CF']
STRUCTURE_THRESHOLD = 10000.0
# --- Añadir junto al resto de constantes ------------------------------------
ADULT_SITE_KEYWORDS = [
    'OnlyFans', 'Fansly', 'Pornhub', 'XVideos', 'Stripchat', 'Chaturbate',
    'ManyVids', 'CamSoda', 'MyFreeCams', 'AdultWork', 'BangBros', 'Brazzers',
    'NaughtyAmerica', 'AmateurPorn', 'Erotica'
]

AD_AGENCY_MCC = {'7311', '7333'}  # Advertising Services / Direct‑mail
FAST_FOOD_HOTEL_PHARMACY_MCC = {'5814', '7011', '5912'}

def generate_anomalous_transactions_ht(
        months: int = 12,
        start_date: str | None = None,
        avg_website_in_per_month: int = 25,
        avg_ad_agency_out_per_month: int = 6,
        avg_expense_spike_per_month: int = 35,
        avg_misc_per_month: int = 10
):
    """
    Genera transacciones sintéticas para un único cliente que financia
    trata de personas. Cubre tres *trigger features* específicas:
        1) Ingresos desde >18 webs “+18”.
        2) Pagos a agencias de publicidad (MCC 7311 / 7333).
        3) Pico de gastos en fast‑food, hoteles y farmacias.
    Devuelve un DataFrame en el mismo esquema que el existente.
    """
    # Punto de partida – idéntico al original ------------------------------
    if start_date is None:
        today = pd.Timestamp(datetime.today().replace(day=1, hour=0, minute=0,
                                                      second=0, microsecond=0))
    else:
        today = pd.to_datetime(start_date).replace(day=1, hour=0, minute=0,
                                                   second=0, microsecond=0)

    periods = [today - pd.DateOffset(months=i) for i in range(months)]
    records = []
    customer_id = 'ANOM001'
    customer_name=faker.name()
    account_id = f"ACC-{customer_id}"

    # ---------------------------------------------------------------------
    for period in periods:
        # ------------- 1. Ingresos “adult websites” ----------------------
        n_web_in = np.random.poisson(lam=avg_website_in_per_month)
        for _ in range(n_web_in):
            ts = period + pd.Timedelta(days=random.randint(0, 27),
                                       hours=random.randint(0, 23),
                                       minutes=random.randint(0, 59),
                                       seconds=random.randint(0, 59))
            website = random.choice(ADULT_SITE_KEYWORDS)
            amt = round(float(np.random.uniform(50, 300)), 2)
            records.append({
                'transaction_id': str(uuid.uuid4()),
                'customer_id': customer_id,
                'customer_name': customer_name,
                'account_id': account_id,
                'transaction_timestamp': ts,
                'transaction_type_code': 'WEB_IN',
                'transaction_type_description': 'Adult‑site payout',
                'channel': 'internet',
                'original_trx_amount': amt,
                'original_trx_currency': 'USD',
                'reference_trx_amount': amt,
                'normalized_country_amount': amt,
                'counterparty_customer_name': website,
                'counterparty_account': faker.iban(),
                'counterparty_contract': '',
                'counterparty_country': 'US',
                'counterparty_country_risk': 'Low',
                'internal': False,
                'in_out': 'IN',
                'merchant_category_code': None,
                'transaction_description': f'Payout from {website}',
                'banker_id': ''
            })

        # ------------- 2. Pagos agencias publicidad ----------------------
        n_ads_out = np.random.poisson(lam=avg_ad_agency_out_per_month)
        for _ in range(n_ads_out):
            ts = period + pd.Timedelta(days=random.randint(0, 27),
                                       hours=random.randint(0, 23),
                                       minutes=random.randint(0, 59),
                                       seconds=random.randint(0, 59))
            mcc = random.choice(tuple(AD_AGENCY_MCC))
            amt = round(float(np.random.uniform(200, 2000)), 2)
            records.append({
                'transaction_id': str(uuid.uuid4()),
                'customer_id': customer_id,
                'customer_name': customer_name,
                'account_id': account_id,
                'transaction_timestamp': ts,
                'transaction_type_code': 'TRF',
                'transaction_type_description': 'Transfer to Advertising',
                'channel': 'internet',
                'original_trx_amount': amt,
                'original_trx_currency': 'USD',
                'reference_trx_amount': amt,
                'normalized_country_amount': amt,
                'counterparty_customer_name': faker.company() + ' Ads',
                'counterparty_account': faker.iban(),
                'counterparty_contract': '',
                'counterparty_country': 'US',
                'counterparty_country_risk': 'Low',
                'internal': False,
                'in_out': 'OUT',
                'merchant_category_code': mcc,
                'transaction_description': 'Advertising payment',
                'banker_id': ''
            })

        # ------------- 3. Pico gastos categorías clave -------------------
        n_exp = np.random.poisson(lam=avg_expense_spike_per_month)
        for _ in range(n_exp):
            ts = period + pd.Timedelta(days=random.randint(0, 27),
                                       hours=random.randint(0, 23),
                                       minutes=random.randint(0, 59),
                                       seconds=random.randint(0, 59))
            mcc = random.choice(tuple(FAST_FOOD_HOTEL_PHARMACY_MCC))
            amt = round(float(np.random.uniform(10, 120)), 2)
            records.append({
                'transaction_id': str(uuid.uuid4()),
                'customer_id': customer_id,
                'customer_name': customer_name,
                'account_id': account_id,
                'transaction_timestamp': ts,
                'transaction_type_code': 'PURCHASE',
                'transaction_type_description': 'POS purchase',
                'channel': 'POS',
                'original_trx_amount': amt,
                'original_trx_currency': 'USD',
                'reference_trx_amount': amt,
                'normalized_country_amount': amt,
                'counterparty_customer_name': faker.company(),
                'counterparty_account': None,
                'counterparty_contract': '',
                'counterparty_country': 'US',
                'counterparty_country_risk': 'Low',
                'internal': False,
                'in_out': 'OUT',
                'merchant_category_code': mcc,
                'transaction_description': f'Purchase MCC {mcc}',
                'banker_id': ''
            })

        # ------------- 4. Ruido “normal” ---------------------------------
        n_misc = np.random.poisson(lam=avg_misc_per_month)
        misc_records = generate_normal_individual_transactions(
            customer_id=customer_id,
            customer_name=faker.name(),
            period=period,
            n_transactions=n_misc
        )
        records.extend(misc_records)

    df = pd.DataFrame(records).sort_values('transaction_timestamp').reset_index(drop=True)
    return df


def calculate_aggregated_features_ht(df_trx, customer_id, customer_name):
    """
    Calcula features mensuales para Human Trafficking.
    Trigger‑features:
        - income_websites_cnt, income_websites_amt
        - ad_agency_out_cnt, ad_agency_out_amt
        - risky_expenses_cnt
    Non‑trigger (normales):
        - cnt_distinct_atm
        - round_amounts
    """
    if df_trx.empty:
        return pd.DataFrame()

    df = df_trx.copy()
    df['year_month'] = df['transaction_timestamp'].dt.to_period('M').dt.to_timestamp()

    out = []
    for period, g in df.groupby('year_month'):
        # --- Trigger 1: ingresos webs “+18”
        web_in = g[g['transaction_type_code'] == 'WEB_IN']
        income_websites_cnt = web_in['counterparty_customer_name'].nunique()
        income_websites_amt = float(web_in['original_trx_amount'].sum())

        # --- Trigger 2: pagos agencias publicidad
        ad_pay = g[g['merchant_category_code'].isin(AD_AGENCY_MCC)]
        ad_agency_out_cnt = int(len(ad_pay))
        ad_agency_out_amt = float(ad_pay['original_trx_amount'].sum())

        # --- Trigger 3: gastos fastfood/hotel/pharmacy
        risky_exp = g[g['merchant_category_code'].isin(FAST_FOOD_HOTEL_PHARMACY_MCC)]
        risky_expenses_cnt = int(len(risky_exp))

        # --- Non‑trigger 1: ATM
        atm_cnt = int((g['transaction_type_code'] == 'ATM').sum())

        # --- Non‑trigger 2: redondeos
        round_cnt = int((g['transaction_type_code'] == 'ROUND_AMOUNT').sum())

        out.append({
            'customer_id': customer_id,
            'customer_name': customer_name,
            'year_month': period,
            'year_month_str': period.strftime('%Y-%m'),

            # Trigger‑features
            'income_websites_cnt': income_websites_cnt,
            'income_websites_amt': income_websites_amt,
            'ad_agency_out_cnt': ad_agency_out_cnt,
            'ad_agency_out_amt': ad_agency_out_amt,
            'risky_expenses_cnt': risky_expenses_cnt,

            # Non‑trigger
            'cnt_distinct_atm': atm_cnt,
            'round_amounts': round_cnt
        })

    return pd.DataFrame(out)
    
def generate_aggregated_dataset_ht(n_customers: int = 100,
                                   months: int = 12,
                                   transactions_per_customer_per_month: int = 20):
    """
    Igual que la versión original, pero el cliente anómalo sigue patrón HT.
    """
    # Preparación de clientes (igual) ------------------------------------
    normal_customers = [f"IND{i:04d}" for i in range(n_customers - 1)]
    anomaly_customer = "ANOM001"
    all_customers = normal_customers + [anomaly_customer]
    customer_names = {c: faker.name() for c in all_customers}

    today = pd.Timestamp(datetime.today().replace(day=1, hour=0, minute=0, second=0,
                                                  microsecond=0))
    periods = [today - pd.DateOffset(months=i) for i in range(months)]

    # -------- 1. Transacciones ------------------------------------------
    tx_records = []

    # normales
    for cust in normal_customers:
        for p in periods:
            tx_records.extend(
                generate_normal_individual_transactions(
                    customer_id=cust,
                    customer_name=customer_names[cust],
                    period=p,
                    n_transactions=transactions_per_customer_per_month
                )
            )

    # anómalo HT
    ht_txs = generate_anomalous_transactions_ht(months=months)
    tx_records.extend(ht_txs.to_dict('records'))

    transactions_df = pd.DataFrame(tx_records)

    # -------- 2. Features agregadas -------------------------------------
    agg_records = []

    for cust in normal_customers:
        for p in periods:
            agg_records.append(
                calculate_normal_aggregated_features(
                    transactions_df, cust, customer_names[cust], p
                )
            )

    ht_features = calculate_aggregated_features_ht(
        ht_txs, anomaly_customer, customer_names[anomaly_customer]
    )
    agg_records.extend(ht_features.to_dict('records'))

    aggregated_df = pd.DataFrame(agg_records)\
        .sort_values(['customer_id', 'year_month'])

    # -------- 3. Salida ordenada ----------------------------------------
    transactions_df = transactions_df.sort_values(['customer_id',
                                                   'transaction_timestamp'])
    return transactions_df, aggregated_df


def generate_normal_individual_transactions(customer_id, customer_name, period, n_transactions=20):
    """
    Genera transacciones normales para clientes individuales regulares
    """
    records = []
    account_id = f"ACC-{customer_id}"
    
    # Generar transacciones distribuidas a lo largo del mes
    for _ in range(n_transactions):
        ts = period + pd.Timedelta(
            days=random.randint(0, 27),
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59)
        )
        
        # Tipos de transacciones normales
        tx_type = random.choice(['ATM', 'TRANSFER', 'DEPOSIT', 'PURCHASE'])
        channel = random.choice(['ATM', 'internet', 'branch', 'mobile'])
        
        # Montos normales (más pequeños que el cliente anómalo)
        if tx_type == 'ATM':
            amt = float(np.random.randint(20, 100) * 10)  # ATM normales más pequeños
        elif tx_type == 'TRANSFER':
            amt = round(float(np.random.uniform(500, 5000)), 2)  # Transferencias normales
        else:
            amt = round(float(np.random.uniform(100, 2000)), 2)  # Otros tipos
        
        # Países de bajo riesgo para transferencias
        if tx_type == 'TRANSFER':
            country = random.choice(['CA', 'GB', 'FR', 'DE', 'AU', 'JP'])
            country_risk = 'Low'
        else:
            country = None
            country_risk = None
        
        records.append({
            'transaction_id': str(uuid.uuid4()),
            'customer_id': customer_id,
            'customer_name': customer_name,
            'account_id': account_id,
            'transaction_timestamp': ts,
            'transaction_type_code': tx_type,
            'transaction_type_description': f'{tx_type} Transaction',
            'channel': channel,
            'original_trx_amount': amt,
            'original_trx_currency': 'USD',
            'reference_trx_amount': amt,
            'normalized_country_amount': amt,
            'counterparty_customer_name': faker.name() if tx_type == 'TRANSFER' else None,
            'counterparty_account': f"ACCT-{faker.bothify('????-########')}" if tx_type == 'TRANSFER' else None,
            'counterparty_contract': '',
            'counterparty_country': country,
            'counterparty_country_risk': country_risk,
            'internal': False,
            'in_out': random.choice(['IN', 'OUT']),
            'transaction_description': f'Normal {tx_type.lower()} transaction',
            'banker_id': f"BNK-{random.randint(1000,9999)}" if channel == 'branch' else ''
        })
    
    return records

def calculate_normal_aggregated_features(transactions_df, customer_id, customer_name, period):
    """
    Calcula features agregadas para clientes normales en el esquema HT.
    - Trigger features (con ruido bajo aleatorio para no alertar):
        income_websites_cnt / income_websites_amt
        ad_agency_out_cnt   / ad_agency_out_amt
        risky_expenses_cnt
    - Non-trigger features (reales):
        cnt_distinct_atm, round_amounts
    """
    # Filtrar transacciones del período mensual específico
    start_date = period
    end_date = period + pd.DateOffset(months=1)
    mask = (transactions_df['transaction_timestamp'] >= start_date) & \
           (transactions_df['transaction_timestamp'] < end_date) & \
           (transactions_df['customer_id'] == customer_id)
    period_txs = transactions_df[mask].copy()

    # --- Non-trigger reales --------------------------------------------------
    atm_cnt = float(int((period_txs['transaction_type_code'] == 'ATM').sum())) if len(period_txs) > 0 else 0.0
    round_cnt = float(int((period_txs['transaction_type_code'] == 'ROUND_AMOUNT').sum())) if len(period_txs) > 0 else 0.0

    # --- Trigger features con ruido bajo (no necesariamente derivadas de txs) -
    # Diseño: muy baja probabilidad de actividad HT en clientes normales
    # Ajusta las probabilidades si quieres más/menos ruido.
    # Income desde webs +18: 0–2 pagos pequeños/mes con prob. baja
    income_cnt = int(np.random.binomial(n=2, p=0.10))          # 10% de prob por intento
    income_amt = float(np.round(np.sum(np.random.uniform(20, 150, size=income_cnt)), 2))

    # Pagos a agencias de publicidad (MCC advertising): 0–1 pagos pequeños/mes
    ad_cnt = int(min(np.random.poisson(lam=0.15), 1))           # media 0.15 => casi siempre 0
    ad_amt = float(np.round(np.sum(np.random.uniform(30, 200, size=ad_cnt)), 2))

    # Gastos en fast-food/hoteles/farmacia: 0–4 (Poisson baja)
    risky_cnt = int(min(np.random.poisson(lam=1.2), 4))

    return {
        'customer_id': customer_id,
        'customer_name': customer_name,
        'year_month': period,
        'year_month_str': period.strftime("%Y-%m"),

        # Trigger features (HT)
        'income_websites_cnt': float(income_cnt),
        'income_websites_amt': income_amt,
        'ad_agency_out_cnt': float(ad_cnt),
        'ad_agency_out_amt': ad_amt,
        'risky_expenses_cnt': float(risky_cnt),

        # Non-trigger (normales)
        'cnt_distinct_atm': atm_cnt,
        'round_amounts': round_cnt
    }



def generate_anomalous_kyc(name):
    """
    Generate KYC record for the anomalous customer.
    Returns a pandas DataFrame with one record.
    """
    dob = faker.date_of_birth(minimum_age=30, maximum_age=70)
    eff_date = faker.date_between(start_date='-5y', end_date='-1y')
    data = {
        'customer_id': 'ANOM001',
        'customer_num': f"NUM-{faker.bothify('######')}",
        'type_of_document': random.choice(['Passport', 'Driver License', 'National ID']),
        'document_number_id_code': faker.bothify('??########'),
        'name': name,
        'date_of_birth': dob,
        'country_of_birth_nationality': 'CU',
        'tax_residency_countries': 'United States',
        'country_of_residence_code': 'US',
        'country_of_residence': 'United States',
        'citizenship_countries_code': 'US',
        'citizenship_countries': 'United States, Cuba',
        'address': faker.address(),
        'phone_number': faker.phone_number(),
        'occupation': random.choice(['Entrepreneur', 'Investor', 'Consultant', 'Executive']),
        'is_unemployed': False,
        'customer_effective_date': eff_date,
        'aml_risk_segment': random.randint(4,6),
        'pep': random.choice([True, False]),
        'segment_type': 'PERS',
        'segment_type_description': 'Personal',
        'sars_flag': False
    }
    return pd.DataFrame([data])

In [11]:
df_trx, agg_df = generate_aggregated_dataset_ht()
dataset_functions.write(context, context.get_spark_session().createDataFrame(agg_df), customer_monthly_dataset().identifier)
anom_name = agg_df.loc[agg_df.customer_id=='ANOM001']['customer_name'].iloc[0]

dataset_functions.write(
    context,
    context.get_spark_session().createDataFrame(df_trx),
    transactions_dataset().identifier
)
dataset_functions.publish(context, transactions_dataset().identifier)

dataset_functions.write(context, context.get_spark_session().createDataFrame(generate_anomalous_kyc(name=anom_name)), customers_dataset().identifier)
dataset_functions.publish(context, customers_dataset().identifier)

2025-07-29 12:49:41,125:INFO:thetaray.common.logging:### DataSet - writing started ###
2025-07-29 12:49:43,832:INFO:thetaray.common.logging:### DataSet - writing done, 1200 written, 0 corrupted, 0 rejected  ###
2025-07-29 12:49:43,993:INFO:thetaray.common.logging:### DataSet - writing started ###
2025-07-29 12:49:47,469:INFO:thetaray.common.logging:### DataSet - writing done, 24658 written, 0 corrupted, 0 rejected  ###
2025-07-29 12:49:50,731:INFO:thetaray.common.logging:finished publishing records for dataset demo_human_trafficking_transactions 
2025-07-29 12:49:50,755:INFO:thetaray.common.logging:### DataSet - writing started ###
2025-07-29 12:49:51,953:INFO:thetaray.common.logging:Partition does not exists, nothing to delete.
2025-07-29 12:49:52,727:INFO:thetaray.common.logging:### DataSet - writing done, 1 written, 0 corrupted, 0 rejected  ###
2025-07-29 12:49:53,339:INFO:thetaray.common.logging:finished publishing records for dataset demo_human_trafficking_customers 


True

In [ ]:
context.close()